# Redes Neurais para Classificação de Objetos

Neste notebook, vamos aprender os fundamentos de **redes neurais artificiais** aplicadas à classificação de objetos. 
Trabalharemos com o dataset **digits** do sklearn, que contém imagens de dígitos de 0 a 9. 

O objetivo é introduzir os conceitos de **MLP (Multilayer Perceptron)**, **pré-processamento**, **treinamento**, e **avaliação de modelos** antes de avançarmos para redes convolucionais em aulas futuras.

## 1. Conceitos teóricos

**Redes Neurais Artificiais (ANN)** são modelos inspirados no funcionamento do cérebro humano. Elas são formadas por **neurônios artificiais** organizados em camadas:

- **Camada de entrada:** recebe os dados brutos (ex.: pixels de uma imagem).
- **Camadas escondidas:** processam os dados aplicando funções de ativação, detectando padrões complexos.
- **Camada de saída:** produz a previsão final, por exemplo, a classe de um dígito.

### Funcionamento de uma camada:
Cada neurônio realiza:
1. Uma **combinação linear** dos inputs: 𝑧 = w1*x1 + w2*x2 + ... + b
2. Passa o resultado por uma **função de ativação** (ex.: ReLU, sigmoid, softmax)

### Aprendizado:
- O modelo ajusta os **pesos (w)** e **bias (b)** para minimizar a diferença entre previsão e valor real.
- Usamos **backpropagation** + **otimizadores** (ex.: Adam) para atualizar os pesos.

### Por que MLP funciona para imagens pequenas
- Cada pixel da imagem é tratado como uma característica independente.
- Para imagens pequenas (8x8), isso é suficiente para classificar os dígitos.
- Limitação: não preserva **informação espacial** (vizinhaça de pixels), mas introduziremos isso mais tarde em CNNs.

## 2. Importando bibliotecas

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

## 3. Carregando e explorando os dados

In [ ]:
digits = load_digits()
X = digits.images
y = digits.target

plt.figure(figsize=(8,4))
for i in range(10):
    plt.subplot(2,5,i+1)
    plt.imshow(X[i], cmap='gray')
    plt.title(y[i])
    plt.axis('off')
plt.show()

## 4. Pré-processamento dos dados

Passos necessários antes de treinar a rede:
1. Achatar imagens 8x8 → vetor de 64 elementos.
2. Normalizar valores dos pixels para melhorar aprendizado.
3. Transformar classes em **one-hot encoding** para usar softmax.

In [ ]:
X_flat = X.reshape((X.shape[0], -1))
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_flat)
y_cat = to_categorical(y, num_classes=10)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_cat, test_size=0.2, random_state=42)

## 5. Construindo a rede neural

Estrutura simples:
- Camada oculta: 32 neurônios, ReLU
- Camada de saída: 10 neurônios, softmax

**Funções de ativação:**
- ReLU: mantém valores positivos e zera negativos, ajuda aprendizado rápido
- Softmax: transforma saídas em probabilidades somando 1

In [ ]:
model = Sequential([
    Dense(32, activation='relu', input_shape=(64,)),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

## 6. Treinando a rede

- Validamos o modelo durante o treinamento para monitorar overfitting.
- Número de épocas: 30
- Batch size: 32

In [ ]:
history = model.fit(X_train, y_train, validation_split=0.2, epochs=30, batch_size=32)

## 7. Avaliando o modelo

Calculamos acurácia no conjunto de teste e visualizamos algumas previsões.

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Acurácia no conjunto de teste: {test_acc:.2f}')

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

plt.figure(figsize=(8,4))
for i in range(10):
    plt.subplot(2,5,i+1)
    plt.imshow(X_test[i].reshape(8,8), cmap='gray')
    plt.title(f'T:{y_true[i]} P:{y_pred_classes[i]}')
    plt.axis('off')
plt.show()

## 8. Conclusão

- Aprendemos a construir uma **rede MLP** para classificação de imagens.
- Revisamos conceitos de **neurônios, camadas, funções de ativação e backpropagation**.
- Vimos a importância do **pré-processamento**, normalização e codificação one-hot.

Em aulas futuras, exploraremos **redes convolucionais (CNNs)**, que conseguem extrair padrões espaciais automaticamente e são mais poderosas para imagens complexas.

# Problema de Classificação com MLP

**Título:** Classificação de Espécies de Animais

**Descrição:**
Desenvolva um modelo de **Rede Neural Perceptron Multicamadas (MLP)** para classificar espécies de animais a partir de suas características físicas, utilizando o **Zoo Dataset** (disponível no Kaggle ou via `sklearn.datasets`). O modelo deve prever corretamente a classe do animal — mamífero, ave, réptil, peixe, anfíbio, inseto ou molusco — com alta acurácia.

**Sobre o Dataset:**

* **Conjunto:** 101 instâncias, 16 atributos (15 booleanos e 1 numérico — número de patas).
* **Classes:** 7 categorias de animais.
* **Acesso:** `sklearn.datasets.fetch_openml(name='zoo', version=1)` ou [Kaggle](https://www.kaggle.com/datasets/uciml/zoo-animal-classification).

**Tarefas:**

1. Carregar e dividir o dataset em **treino (80%)** e **teste (20%)**.
2. Implementar uma **MLP** com:

   * 1–2 camadas ocultas (10–50 neurônios cada).
   * Ativação **ReLU** (ocultas) e **Softmax** (saída).
   * Função de perda: **Entropia Cruzada Categórica**.
   * Otimizador: **Adam**.
3. Avaliar o desempenho com **acurácia, precisão, recall e F1-score**.
4. Ajustar **hiperparâmetros** (taxa de aprendizado, número de épocas) e aplicar **regularização** (ex.: dropout) para reduzir overfitting.
5. Gerar uma **matriz de confusão** para análise detalhada das classes.

**Desafios Extras:**

* Tratar o **desbalanceamento de classes**.
* **Normalizar** o atributo numérico (número de patas).
* Aplicar **validação cruzada** para garantir robustez.

**Resultado Esperado:**

* Modelo com **acurácia acima de 90%**.
* Relatório final com métricas e matriz de confusão.